# 🤖 헤드헌터 AI 에이전트 데모

헤드헌터 AI 에이전트의 핵심 기능들을 테스트해보는 데모 노트북입니다.

In [ ]:
# 환경 설정
import sys
import os
from pathlib import Path

# 프로젝트 루트를 Python 경로에 추가
project_root = Path().resolve().parent
sys.path.append(str(project_root))

# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

print(f"프로젝트 루트: {project_root}")
print("환경 변수 로드 완료")

## 📊 데이터베이스 연결 테스트

In [ ]:
# PostgreSQL 연결 테스트
try:
    from src.database.repositories import get_candidate_repository
    
    repo = get_candidate_repository()
    stats = repo.get_statistics()
    
    print("✅ PostgreSQL 연결 성공!")
    print(f"📊 총 인재 수: {stats['total_candidates']}명")
    print(f"📍 지역 분포: {len(stats['location_distribution'])}개 지역")
    print(f"🛠️ 인기 스킬 TOP 5: {[skill['skill'] for skill in stats['top_skills'][:5]]}")
    
except Exception as e:
    print(f"❌ PostgreSQL 연결 실패: {e}")
    print("💡 docker-compose up -d 명령어로 PostgreSQL을 시작해주세요.")

## 🔍 벡터 스토어 테스트

In [ ]:
# FAISS 벡터 스토어 테스트
try:
    from src.vector_store.faiss_store import get_vector_store
    
    vector_store = get_vector_store()
    stats = vector_store.get_stats()
    
    print("✅ FAISS 벡터 스토어 연결 성공!")
    print(f"📚 총 문서 수: {stats['total_documents']}개")
    print(f"📂 카테고리별 분포:")
    for category, count in stats['categories'].items():
        print(f"   • {category}: {count}개")
        
    # 샘플 검색 테스트
    print("\n🔍 샘플 검색 테스트: 'Python 기술'")
    results = vector_store.search("Python 기술 특징", top_k=2)
    
    for i, result in enumerate(results, 1):
        print(f"   {i}. {result['metadata'].get('title', 'N/A')} (점수: {result['score']:.3f})")
        print(f"      내용: {result['content'][:100]}...")
    
except Exception as e:
    print(f"❌ 벡터 스토어 연결 실패: {e}")
    print("💡 python setup_project.py 명령어로 벡터 스토어를 초기화해주세요.")

## 🛠️ 도구 기능 테스트

In [ ]:
# 정형 데이터 도구 테스트
print("=== 정형 데이터 도구 테스트 (PostgreSQL) ===")

try:
    from src.tools.candidate_tools import search_candidates_by_skills
    
    # Python 개발자 검색
    result = search_candidates_by_skills.invoke({"skills": ["Python"], "min_experience": 3})
    
    if result.get('success'):
        print(f"✅ {result['message']}")
        print(f"📊 검색 결과: {result['count']}명")
        
        if result['candidates']:
            candidate = result['candidates'][0]
            print(f"🧑‍💻 첫 번째 후보자: {candidate['name']} ({candidate['location']})")
    else:
        print(f"❌ 검색 실패: {result.get('message')}")
        
except Exception as e:
    print(f"❌ 정형 데이터 도구 테스트 실패: {e}")

In [ ]:
# 비정형 데이터 도구 테스트
print("=== 비정형 데이터 도구 테스트 (FAISS Vector DB) ===")

try:
    from src.tools.market_tools import search_tech_information
    
    # Python 기술 정보 검색
    result = search_tech_information.invoke({"technology": "Python", "top_k": 2})
    
    if result.get('success'):
        print(f"✅ {result['message']}")
        print(f"📊 검색 결과: {result['count']}건")
        
        if result['information']:
            info = result['information'][0]
            print(f"📄 첫 번째 정보: {info['metadata'].get('title', 'N/A')}")
            print(f"💬 내용: {info['content'][:150]}...")
    else:
        print(f"❌ 검색 실패: {result.get('message')}")
        
except Exception as e:
    print(f"❌ 비정형 데이터 도구 테스트 실패: {e}")

## 🌐 웹 검색 도구 테스트

In [ ]:
# 웹 검색 도구 테스트 (Tavily API 키 필요)
print("=== 웹 검색 도구 테스트 (Tavily) ===")

try:
    from src.tools.web_search_tools import web_search_latest_trends
    
    # 최신 개발자 트렌드 검색
    result = web_search_latest_trends.invoke({
        "query": "2024 개발자 트렌드", 
        "max_results": 2
    })
    
    if result.get('success'):
        print(f"✅ {result['message']}")
        print(f"📊 검색 결과: {result['count']}건")
        
        if result['results']:
            news = result['results'][0]
            print(f"📰 첫 번째 뉴스: {news['title']}")
            print(f"🔗 URL: {news['url']}")
            print(f"💬 내용: {news['content'][:150]}...")
    else:
        print(f"❌ 검색 실패: {result.get('message')}")
        
except Exception as e:
    print(f"❌ 웹 검색 도구 테스트 실패: {e}")
    print("💡 .env 파일에 TAVILY_API_KEY를 설정해주세요.")

## 🤖 AI 에이전트 워크플로우 테스트

In [ ]:
# AI 에이전트 워크플로우 테스트
print("=== AI 에이전트 워크플로우 테스트 ===")

try:
    from src.agents.workflow import get_headhunter_workflow
    from langchain_core.messages import HumanMessage
    
    # 워크플로우 초기화
    workflow = get_headhunter_workflow()
    graph = workflow.get_graph()
    
    print("✅ 워크플로우 초기화 성공!")
    
    # 워크플로우 구조 확인
    graph_info = graph.get_graph()
    print(f"📊 워크플로우 노드 수: {len(graph_info.nodes)}개")
    print(f"🔗 워크플로우 엣지 수: {len(graph_info.edges)}개")
    
    print("\n📋 워크플로우 노드 목록:")
    for node in graph_info.nodes:
        print(f"   • {node}")
    
    # 간단한 워크플로우 실행 테스트 (주의: 실제 API 호출)
    test_query = "안녕하세요, 헤드헌터 AI 에이전트입니다."
    print(f"\n🧪 테스트 쿼리: {test_query}")
    print("⚠️  실제 API 호출을 피하기 위해 구조 확인만 진행합니다.")
    
except Exception as e:
    print(f"❌ 워크플로우 테스트 실패: {e}")
    print("💡 .env 파일에 UPSTAGE_API_KEY를 설정해주세요.")

## 📊 전체 시스템 상태 요약

In [ ]:
# 전체 시스템 상태 요약
print("=== 🎯 헤드헌터 AI 에이전트 시스템 상태 요약 ===")

components = {
    "PostgreSQL (정형 데이터)": False,
    "FAISS Vector DB (비정형 데이터)": False,
    "AI 워크플로우 (LangGraph)": False,
    "정형 데이터 도구들": False,
    "비정형 데이터 도구들": False,
    "웹 검색 도구들": False
}

# 각 컴포넌트 상태 확인
try:
    from src.database.repositories import get_candidate_repository
    repo = get_candidate_repository()
    stats = repo.get_statistics()
    if stats['total_candidates'] > 0:
        components["PostgreSQL (정형 데이터)"] = True
        components["정형 데이터 도구들"] = True
except:
    pass

try:
    from src.vector_store.faiss_store import get_vector_store
    vector_store = get_vector_store()
    stats = vector_store.get_stats()
    if stats['total_documents'] > 0:
        components["FAISS Vector DB (비정형 데이터)"] = True
        components["비정형 데이터 도구들"] = True
except:
    pass

try:
    from src.agents.workflow import get_headhunter_workflow
    workflow = get_headhunter_workflow()
    graph = workflow.get_graph()
    components["AI 워크플로우 (LangGraph)"] = True
except:
    pass

# API 키 확인
try:
    import os
    if os.getenv('TAVILY_API_KEY'):
        components["웹 검색 도구들"] = True
except:
    pass

# 상태 출력
print("\n📊 시스템 컴포넌트 상태:")
for component, status in components.items():
    icon = "✅" if status else "❌"
    print(f"   {icon} {component}")

# 활성화된 컴포넌트 수
active_count = sum(components.values())
total_count = len(components)

print(f"\n🎯 전체 시스템 준비도: {active_count}/{total_count} ({active_count/total_count*100:.1f}%)")

if active_count == total_count:
    print("\n🎉 모든 시스템이 정상 작동 중입니다! Streamlit 앱을 실행해보세요.")
    print("   명령어: python streamlit_run.py")
else:
    print("\n⚠️  일부 시스템에 문제가 있습니다. 다음을 확인해주세요:")
    if not components["PostgreSQL (정형 데이터)"]:
        print("   • docker-compose up -d (PostgreSQL 시작)")
        print("   • python setup_project.py (데이터베이스 초기화)")
    if not components["FAISS Vector DB (비정형 데이터)"]:
        print("   • python setup_project.py (벡터 스토어 초기화)")
    if not components["웹 검색 도구들"]:
        print("   • .env 파일에 TAVILY_API_KEY 설정")
    if not components["AI 워크플로우 (LangGraph)"]:
        print("   • .env 파일에 UPSTAGE_API_KEY 설정")

## 🎯 다음 단계

데모 테스트가 완료되면 다음 단계를 진행하세요:

1. **Streamlit 앱 실행**: `python streamlit_run.py`
2. **AI 상담 테스트**: 예시 질문들로 에이전트 기능 확인
3. **워크플로우 시각화**: 새로 추가된 "🔄 워크플로우" 탭에서 시각화 확인
4. **도구 기능 테스트**: README의 29개 예시 질문으로 기능 검증

### 🧪 테스트용 예시 질문들

**정형 데이터 (PostgreSQL):**
- "Python 개발자 5년 이상 경력자를 찾아줘"
- "강남구에 거주하는 React 개발자 리스트를 보여줘"
- "연봉 8천만원 이상 희망하는 백엔드 개발자가 몇 명이야?"

**비정형 데이터 (FAISS Vector DB):**
- "최근 AI 개발자 시장 트렌드가 어때?"
- "클라우드 엔지니어의 평균 연봉은 얼마나 돼?"
- "스타트업에서 요구하는 핵심 기술스택이 뭐야?"

**웹 검색 (Tavily):**
- "2024년 개발자 채용 시장 최신 동향을 웹에서 찾아줘"
- "네이버에서 현재 채용중인 개발자 포지션이 뭐가 있어?"